In [1]:
#Loading data from the imdb data set and printing

import pandas as pd

name_basics = pd.read_csv("data/name.basics.tsv", sep="\t")
title_basics = pd.read_csv("data/title.basics.tsv", sep="\t")
title_crew = pd.read_csv("data/title.crew.tsv", sep="\t")
title_principals = pd.read_csv("data/title.principals.tsv", sep="\t")
title_ratings = pd.read_csv("data/title.ratings.tsv", sep="\t")

print(name_basics.head())
print(title_basics.head())
print(title_crew.head())
print(title_principals.head())
print(title_ratings.head())

      nconst      primaryName birthYear deathYear  \
0  nm0000001     Fred Astaire      1899      1987   
1  nm0000002    Lauren Bacall      1924      2014   
2  nm0000003  Brigitte Bardot      1934        \N   
3  nm0000004     John Belushi      1949      1982   
4  nm0000005   Ingmar Bergman      1918      2007   

                    primaryProfession                           knownForTitles  
0        actor,miscellaneous,producer  tt0072308,tt0050419,tt0027125,tt0025164  
1  actress,soundtrack,archive_footage  tt0037382,tt0075213,tt0038355,tt0117057  
2   actress,music_department,producer  tt0057345,tt0049189,tt0056404,tt0054452  
3       actor,writer,music_department  tt0072562,tt0077975,tt0080455,tt0078723  
4               writer,director,actor  tt0050986,tt0069467,tt0050976,tt0083922  
      tconst titleType            primaryTitle           originalTitle  \
0  tt0000001     short              Carmencita              Carmencita   
1  tt0000002     short  Le clown et ses chiens 

In [2]:
# Checking the columns of each DataFrame to see which ones are relevant for our analysis
print(name_basics.columns)
print(title_basics.columns)
print(title_crew.columns)
print(title_principals.columns)
print(title_ratings.columns)

Index(['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession',
       'knownForTitles'],
      dtype='object')
Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')
Index(['tconst', 'directors', 'writers'], dtype='object')
Index(['tconst', 'ordering', 'nconst', 'category', 'job', 'characters'], dtype='object')
Index(['tconst', 'averageRating', 'numVotes'], dtype='object')


In [3]:
# Merging the DataFrames to create a comprehensive movies DataFrame
movies_df = pd.merge(title_basics, title_ratings, on="tconst", how="inner")
print(movies_df.head())

      tconst titleType            primaryTitle           originalTitle  \
0  tt0000001     short              Carmencita              Carmencita   
1  tt0000002     short  Le clown et ses chiens  Le clown et ses chiens   
2  tt0000003     short            Poor Pierrot          Pauvre Pierrot   
3  tt0000004     short             Un bon bock             Un bon bock   
4  tt0000005     short        Blacksmith Scene        Blacksmith Scene   

   isAdult startYear endYear runtimeMinutes                    genres  \
0        0      1894      \N              1         Documentary,Short   
1        0      1892      \N              5           Animation,Short   
2        0      1892      \N              5  Animation,Comedy,Romance   
3        0      1892      \N             12           Animation,Short   
4        0      1893      \N              1                     Short   

   averageRating  numVotes  
0            5.7      2168  
1            5.5       298  
2            6.5      2229  


In [4]:
# Cleaning the DataFrame to remove irrelevant or low-quality data

movies_df = movies_df[movies_df["titleType"].isin(["movie"])]
movies_df = movies_df[movies_df["numVotes"] > 1000]
movies_df = movies_df[movies_df["genres"] != "\\N"]
movies_df["startYear"] = pd.to_numeric(movies_df["startYear"], errors='coerce')
movies_df["runtimeMinutes"] = pd.to_numeric(movies_df["runtimeMinutes"], errors='coerce')
movies_df.dropna(subset=["startYear", "runtimeMinutes"], inplace=True)
movies_df = movies_df[movies_df["isAdult"] == 0]
movies_df = movies_df.drop(columns=["endYear","originalTitle"])

print(movies_df.head())

         tconst titleType                               primaryTitle  isAdult  \
377   tt0000574     movie                The Story of the Kelly Gang        0   
1033  tt0002130     movie                            Dante's Inferno        0   
1139  tt0002423     movie                                    Passion        0   
1250  tt0002844     movie  Fantômas: In the Shadow of the Guillotine        0   
1281  tt0003014     movie                              Ingeborg Holm        0   

      startYear  runtimeMinutes                      genres  averageRating  \
377        1906            70.0  Action,Adventure,Biography            6.0   
1033       1911            71.0     Adventure,Drama,Fantasy            7.0   
1139       1919           113.0     Biography,Drama,Romance            6.6   
1250       1913            54.0                 Crime,Drama            6.9   
1281       1913            96.0                       Drama            7.0   

      numVotes  
377       1013  
1033      

In [5]:
# Merging the movies DataFrame with the title_crew DataFrame to add Directors and Writers columns
movies_with_crew_df = pd.merge(movies_df,title_crew,on="tconst",how="inner")
print(movies_with_crew_df.head())


      tconst titleType                               primaryTitle  isAdult  \
0  tt0000574     movie                The Story of the Kelly Gang        0   
1  tt0002130     movie                            Dante's Inferno        0   
2  tt0002423     movie                                    Passion        0   
3  tt0002844     movie  Fantômas: In the Shadow of the Guillotine        0   
4  tt0003014     movie                              Ingeborg Holm        0   

   startYear  runtimeMinutes                      genres  averageRating  \
0       1906            70.0  Action,Adventure,Biography            6.0   
1       1911            71.0     Adventure,Drama,Fantasy            7.0   
2       1919           113.0     Biography,Drama,Romance            6.6   
3       1913            54.0                 Crime,Drama            6.9   
4       1913            96.0                       Drama            7.0   

   numVotes                      directors                        writers  
0   

In [6]:
# Cleaning title_principals DataFrame to only include actors/actresses and then make a DF including only the top 3 for each movie
actors_df = title_principals[title_principals["category"].isin(["actor", "actress"])]
top_actors_df = actors_df[actors_df["ordering"] <= 3]
actor_pivot_df = top_actors_df.pivot_table(index="tconst", columns="ordering", values="nconst", aggfunc="first")
actor_pivot_df.rename(columns={1: "actor_1", 2: "actor_2", 3: "actor_3"}, inplace=True)
actor_pivot_df.reset_index(inplace=True)
print(actor_pivot_df.head())

ordering     tconst    actor_1    actor_2    actor_3
0         tt0000005  nm0443482  nm0653042        NaN
1         tt0000007  nm0179163  nm0183947        NaN
2         tt0000008  nm0653028        NaN        NaN
3         tt0000009  nm0063086  nm0183823  nm1309758
4         tt0000011  nm3692297        NaN        NaN


In [7]:
# Final DataFrame with all relevant features
final_df = pd.merge(movies_with_crew_df, actor_pivot_df, on="tconst", how="left")
final_df = final_df.drop(columns=["isAdult","titleType","primaryTitle"])
print(final_df.head())

      tconst  startYear  runtimeMinutes                      genres  \
0  tt0000574       1906            70.0  Action,Adventure,Biography   
1  tt0002130       1911            71.0     Adventure,Drama,Fantasy   
2  tt0002423       1919           113.0     Biography,Drama,Romance   
3  tt0002844       1913            54.0                 Crime,Drama   
4  tt0003014       1913            96.0                       Drama   

   averageRating  numVotes                      directors  \
0            6.0      1013                      nm0846879   
1            7.0      3879  nm0078205,nm0655824,nm0209738   
2            6.6      1087                      nm0523932   
3            6.9      2655                      nm0275421   
4            7.0      1549                      nm0803705   

                         writers    actor_1    actor_2    actor_3  
0                      nm0846879  nm0846887  nm0846894  nm1431224  
1                      nm0019604  nm0660139  nm0685283  nm0209738  
2 

In [9]:
# Saving file as parquet to use in model training
final_df.to_parquet("data/final_movie_data.parquet")